Left-over code:

<table><tr>
<td> <img src="BiPlane_Lift_Chord_vs_Gap.JPG" height="500" width="700"> </td>
<td> <img src="Monoplane_vs_Biplane_vs_Triplane.JPG" height="400" width="700"> </td>
</tr></table>

<h3 align = 'center' color=#CC0000>**Figure 1:** Lift vs Gap/Chord (Biplanes) &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;**Figure 2:** Lift vs Angle of Attack (Biplanes & Triplanes)</h3>

In [ ]:
# plot each geometry
width = 5
pyplot.figure(figsize=(width*1.5, width))
pyplot.title('Biplane Geometry (Gap = 0.75)')
pyplot.grid()
pyplot.xlabel('x', fontsize=16)
pyplot.ylabel('y', fontsize=16)
pyplot.plot(x_m, y_m, color='k', linestyle='-', linewidth=2)
pyplot.plot(x_bi_4, y_bi_4, color='k', linestyle='-', linewidth=2)
pyplot.xlim(-0.1, 1.1)
pyplot.ylim(-0.2, 1.0);

In [ ]:

#----------------------------------------------------------------------#

# gap 2: 1.00
A_bi_2 = build_singularity_matrix(A_source_bi_2, B_vortex_bi_2, N)
b_bi4_2 = build_freestream_rhs(panelS_bi_2, freestream_4, N)
#print(A_bi_2)
#print(b_bi4_2)

# solve for the singularity matrices
strengths_bi4_2 = numpy.linalg.solve(A_bi_2,b_bi4_2)

#store the strengths on each panel
for i, panel in enumerate(panelS_bi_2):
    panel.sigma = strengths_bi4_2[i]

# store the circulation density
gamma_bi4_2 = strengths_bi4_2[-2:]

#----------------------------------------------------------------------#

# gap 3: 1.25
A_bi_3 = build_singularity_matrix(A_source_bi_3, B_vortex_bi_3, N)
b_bi4_3 = build_freestream_rhs(panelS_bi_3, freestream_4, N)
#print(A_bi_3)
#print(b_bi4_3)

# solve for the singularity matrices
strengths_bi4_3 = numpy.linalg.solve(A_bi_3,b_bi4_3)

#store the strengths on each panel
for i, panel in enumerate(panelS_bi_3):
    panel.sigma = strengths_bi4_3[i]

# store the circulation density
gamma_bi4_3 = strengths_bi4_3[-2:]

#----------------------------------------------------------------------#

# gap 4: 1.50
A_bi_4 = build_singularity_matrix(A_source_bi_4, B_vortex_bi_4, N)
b_bi4_4 = build_freestream_rhs(panelS_bi_4, freestream_4, N)
#print(A_bi_4)
#print(b_bi4_4)

### Flow Tangency Boundary Condition

Now we will incorporate the flow tangency boundary condition. This will make the body geometry correspond to a dividing streamline, as flow is not permitted to penetrate through the panels. To do this for each panel $i$, we make $u_n=0$ at $(x_{c_i},y_{c_i})$, and we will add a source strength $\sigma$ and a vortex strength $\gamma$ to each panel. This will lead to:

$$
\begin{split}
    \phi(x, y) &= U_\infty x \cos\alpha + U_\infty y \sin\alpha \\
    & + \int_{lower} \frac{\sigma(s)}{2 \pi} \ln \sqrt{(x - \xi(s))^2 + (y - \eta(s))^2} ds  \\
    & + \int_{upper} \frac{\sigma(s)}{2 \pi} \ln \sqrt{(x - \xi(s))^2 + (y - \eta(s))^2} ds \\
        & - \int_{lower} \frac{\gamma(s)}{2 \pi} \tan^{-1} \left( \frac{y - \eta(s)}{x - \xi(s)} \right) ds  \\
    & - \int_{upper} \frac{\gamma(s)}{2 \pi} \tan^{-1} \left( \frac{y - \eta(s)}{x - \xi(s)} \right) ds
\end{split}
$$

Here: 
1. Each panel on each airfoil will have a different source strength $\sigma$.
2. All panels for a given airfoil will share the same vortex strength $\gamma$ for simplicity.
3. $\sigma(s)$ and $\gamma(s)$ can be taken as constants for each panel, as they won't change over the length of the panel.

i.e.: 
$$
\begin{align*}
0 &= V_\infty \cos \left(\alpha-\beta_i\right) + \frac{\sigma_i}{2} \\
&+ \sum_{j=1,j\neq i}^N \frac{\sigma_j}{2\pi} \int_j \frac{\partial}{\partial n_i} \ln \left(\sqrt{(x_{c_i}-x_j(s_j))^2+(y_{c_i}-y_j(s_j))^2} \right) {\rm d}s_j \\
&- \sum_{j=1,j\neq i}^N \frac{\gamma}{2\pi} \int_j \frac{\partial}{\partial n_i} \tan^{-1} \left(\frac{y_{c_i}-y_j(s_j)}{x_{c_i}-x_j(s_j)}\right) {\rm d}s_j
\end{align*}
$$

with: $$\frac{\partial x_{c_i}}{\partial t_i} = -\sin \beta_i$$  

$$\frac{\partial y_{c_i}}{\partial t_i} = \cos \beta_i$$

$$
A^n_{ij} = \frac{1}{2 \pi} \int_0^{l_j} \frac{\left( {x_c}_i - \xi_j(s) \right) \cos\beta_i + \left( {y_c}_i - \eta_j(s) \right) \sin\beta_i}{\left( {x_c}_i - \xi_j(s) \right)^2 + \left( {y_c}_i - \eta_j(s) \right)^2} ds
$$

$$
B^n_{ij} = \frac{1}{2 \pi} \int_0^{l_j} \frac{\left( {y_c}_i - \eta_j(s) \right) \cos\beta_i - \left( {x_c}_i - \xi_j(s) \right) \sin\beta_i}{\left( {x_c}_i - \xi_j(s) \right)^2 + \left( {y_c}_i - \eta_j(s) \right)^2} ds
$$

Let's create functions for the integral, and the source and vortex contributions.

In [ ]:
data = numpy.array([[L_bi4_1, L_bi6_1, L_bi8_1, L_bi10_1],
                   [L_bi4_2, L_bi6_2, L_bi8_2, L_bi10_2],
                   [L_bi4_3, L_bi6_3, L_bi8_3, L_bi10_3],
                   [L_bi4_4, L_bi6_4, L_bi8_4, L_bi10_4]])
row_header = ['0.75','1.00','1.25','1.50']
column_header = ([['AOA','','',''],['4 deg','6 deg','8 deg','10 deg']])
df = pandas.DataFrame(data, row_header, column_header)
df.index.name = 'Gap/Chord'
df

In [ ]:
# find Kutta Joukowski lift for the monoplane
Lm_5 = 2*alt_monplane(lower_panels_bi, freestream_4, N)
Lm_6 = 2*alt_monplane(lower_panels_bi, freestream_6, N)
Lm_7 = 2*alt_monplane(lower_panels_bi, freestream_8, N)
Lm_8 = 2*alt_monplane(lower_panels_bi, freestream_10, N)

In [ ]:
L_m4=(Lm_5+Lm_1)/2
L_m6=(Lm_6+Lm_2)/2
L_m8=(Lm_7+Lm_3)/2
L_m10=(Lm_8+Lm_4)/2

'''
L_m4=Lm_1
L_m6=Lm_2
L_m8=Lm_3
L_m10=Lm_4
''';

In [ ]:
# foil lift array
y_n10 = 3.25
n = int((y_n10-y_n4)/.25)
y_n = numpy.linspace(y_n4, y_n10, num=n)
Y_n = numpy.empty(n, dtype=float)
for i, j in enumerate(y_n):
    Y_n = translate_geo(x_m, y_m, x_n1, y_n[i])
    


In [ ]:
x_bi_5, y_bi_5 = translate_geo(x_m, y_m, x_n1, y_n[0])
x_bi_5, y_bi_5 = translate_geo(x_m, y_m, x_n1, y_n[1])
x_bi_7, y_bi_7 = translate_geo(x_m, y_m, x_n1, y_n[2])
x_bi_8, y_bi_8 = translate_geo(x_m, y_m, x_n1, y_n[3])
x_bi_9, y_bi_9 = translate_geo(x_m, y_m, x_n1, y_n[4])
x_bi_10, y_bi_10 = translate_geo(x_m, y_m, x_n1, y_n[5])
x_bi_11, y_bi_11 = translate_geo(x_m, y_m, x_n1, y_n[6])

In [ ]:
# discretize each of the upper airfoils
# gap 5 (1.75) - gap 11 (3.25)
upper_panels_bi_5 = define_panels(x_bi_5, y_bi_5, N-1)
upper_panels_bi_6 = define_panels(x_bi_6, y_bi_6, N-1)
upper_panels_bi_7 = define_panels(x_bi_7, y_bi_7, N-1)
upper_panels_bi_8 = define_panels(x_bi_8, y_bi_8, N-1)
upper_panels_bi_9 = define_panels(x_bi_9, y_bi_9, N-1)
upper_panels_bi_10 = define_panels(x_bi_10, y_bi_10, N-1)
upper_panels_bi_11 = define_panels(x_bi_11, y_bi_11, N-1)

# combine the panelS arrays:
# gap 5 (1.75) - gap 11 (3.25)
panelS_bi_5=numpy.concatenate((lower_panels_bi, upper_panels_bi_5))
panelS_bi_6=numpy.concatenate((lower_panels_bi, upper_panels_bi_6))
panelS_bi_7=numpy.concatenate((lower_panels_bi, upper_panels_bi_7))
panelS_bi_8=numpy.concatenate((lower_panels_bi, upper_panels_bi_8))
panelS_bi_9=numpy.concatenate((lower_panels_bi, upper_panels_bi_9))
panelS_bi_10=numpy.concatenate((lower_panels_bi, upper_panels_bi_10))
panelS_bi_11=numpy.concatenate((lower_panels_bi, upper_panels_bi_11))

# solve for the lift
# Freestream = 6 deg
# gap 5: 1.75
L_bi6_5 = solve_biplane(panelS_bi_5, freestream_6, N)
# gap 6: 2.00
L_bi6_6 = solve_biplane(panelS_bi_6, freestream_6, N)
# gap 7: 2.25
L_bi6_7 = solve_biplane(panelS_bi_7, freestream_6, N)

In [ ]:
data.shape = (1,8)
column_header = ([['Gap/Chord','','','','','','','' ],\
                  ['1.75','2.00','2.25', '2.50', '2.75', '3.00', '3.25', '3.5']])
row_header = (['AOA = 6 degrees'])
df = pandas.DataFrame(data, row_header ,column_header)
#df.index.name = 'Gap/Chord'
df

In [ ]:
print(N-1)
print(((N-1)/3))
print((1+(N-1)/3))
print((2*(N-1)/3))
print(1+(2*(N-1)/3))

In [ ]:
q=numpy.linspace(1,33,num=33)
#print(q)
x1 = q[:int((N-1)/3)]
x2 = q[int((N-1)/3):int(2*(N-1)/3)]
x3 = q[int(2*(N-1)/3):]
print(x1)
print(x2)
print(x3)
print(len(x1), len(x2), len(x3))

In [ ]:
top_angle1 = [panel.beta for panel in panelS_tri[88:]]
top_angle2 = [panel.beta for panel in panelS_tri]
print(len(top_angle1))
print(len(top_angle2))
print(top_angle2)
print(top_angle1)

In [ ]:
Q=numpy.linspace(1,200,num=200)
N=50
x1 = Q[:100]
x2 = Q[100:]
print(x1)
print(x2)
print(len(x1))
print(len(x2))

In [2]:
import numpy
Q=numpy.linspace(1,99,num=99)
#print(Q)
N = int(len(Q)/3)
print(N)
x1 = Q[:N]
x2 = Q[N:-N]
x3 = Q[-N:]
print(x1)
print(x2)
print(x3)
print(len(x1))
print(len(x2))
print(len(x3))

33
[ 1.  2.  3.  4.  5.  6.  7.  8.  9. 10. 11. 12. 13. 14. 15. 16. 17. 18.
 19. 20. 21. 22. 23. 24. 25. 26. 27. 28. 29. 30. 31. 32. 33.]
[34. 35. 36. 37. 38. 39. 40. 41. 42. 43. 44. 45. 46. 47. 48. 49. 50. 51.
 52. 53. 54. 55. 56. 57. 58. 59. 60. 61. 62. 63. 64. 65. 66.]
[67. 68. 69. 70. 71. 72. 73. 74. 75. 76. 77. 78. 79. 80. 81. 82. 83. 84.
 85. 86. 87. 88. 89. 90. 91. 92. 93. 94. 95. 96. 97. 98. 99.]
33
33
33


In [ ]:
print(len(panelS_tri))
N=int(len(panelS_tri)/3)
print(N)

In [ ]:
# what is the value of the lift for the system (main + flap)
# assume that P_inf = 0, density = 1 => use bernouli's equation
P_inf=0
den=1
u_inf=1.0
N = 
num1 = int(((N-1)/3))
num2 = int((2*(N-1)/3))
print(num2)
# the parts needed for the integral of of lift are:

# tangential velocity of the panels
bottom_vt = [panel.vt for panel in panelS_tri[:num1]]
middle_vt = [panel.vt for panel in panelS_tri[num1:num2]]
top_vt = [panel.vt for panel in panelS_tri[num2:]]

# length of the panels
bottom_len = [panel.length for panel in panelS_tri[:num1]]
middle_len = [panel.length for panel in panelS_tri[num1:num2]]
top_len = [panel.length for panel in panelS_tri[num2:]]
print(len(top_len))
print(len(middle_len))
print(len(bottom_len))

# angle of the panels called n*j (beta)
bottom_angle = [panel.beta for panel in panelS_tri[:num1]]
middle_angle = [panel.beta for panel in panelS_tri[num1:num2]]
top_angle = [panel.beta for panel in panelS_tri[num2:]]
print(len(top_angle))
print(len(middle_angle))
print(len(bottom_angle))
print(top_angle)
print(top_angle[11])

# need to find the lift of the center of each panel individually and add them up

# foil lift array
loquat = numpy.empty(num1, dtype=float)
for i in range (num1):
    loquat[i]=-(P_inf+0.5*den*(u_inf**2-bottom_vt[i]**2))*bottom_len[i]*math.sin(bottom_angle[i])
        
# flap lift array
mango = numpy.empty(num1, dtype=float)
for i in range (num1):
    mango[i]=-(P_inf+0.5*den*(u_inf**2-middle_vt[i]**2))*middle_len[i]*math.sin(middle_angle[i])
      
# flap lift array
plant = numpy.empty(num1, dtype=float)
for i in range (num1):
    plant[i]=-(P_inf+0.5*den*(u_inf**2-top_vt[i]**2))*top_len[i]*math.sin(top_angle[i])
    
#print(loquat)
#print(mango)
#print(plant)
    
# summing the lift arrays and ading them up
L=numpy.sum(loquat)+numpy.sum(mango)+numpy.sum(plant)
#print(L)

In [ ]:
l1 = gamma[0] * sum(panel.length for panel in panelS[:N])
print(l1)
l2 = gamma[1] * sum(panel.length for panel in panelS[N:-N])
print(l2)
l3 = gamma[2] * sum(panel.length for panel in panelS[-N:])
print(l3)
Lift_test = l1+l2+l3
print('alternative lift: '+str(Lift_test))

In [ ]:
  
    # find the source and vortex contributions
    A_source = source_contribution_normal(panelS_tri)
    B_vortex = vortex_contribution_normal(panelS_tri)
    
    # build the singularity matrix
    A = build_singularity_matrix_tri(A_source, B_vortex, N)
    b = build_freestream_rhs_tri(panelS_tri, freestream_4, N)
    
    # solve for the singularity matrices
    strengths = numpy.linalg.solve(A,b)
    
    #store the strengths on each panel
    for i, panel in enumerate(panelS_tri):
        panel.sigma = strengths[i]
        
    # store the circulation density
    gamma = strengths[-3:]
    
    # tangential velocity at each panel center.
    compute_tangential_velocity_tri(panelS_tri, freestream_4, gamma, A_source, B_vortex, N)
    
    # surface pressure coefficient
    compute_pressure_coefficient_tri(panelS_tri, freestream_4)
    
    # check that the work so far is correct => for a closed body the sum of the strengths must = 0
    accuracy = sum([panel.sigma*panel.length for panel in panelS_tri])
    print('sum of singularity strengths: {:0.6f}'.format(accuracy))
    
    # what is the value of the lift for the system (main + flap)
    # assume that P_inf = 0, density = 1 => use bernouli's equation
    P_inf=0
    den=1
    u_inf=1.0
    
    # the parts needed for the integral of of lift are:
    
    # tangential velocity of the panels
    bottom_vt = [panel.vt for panel in panelS_tri[:N]]
    middle_vt = [panel.vt for panel in panelS_tri[N:-N]]
    top_vt = [panel.vt for panel in panelS_tri[-N:]]
    
    # length of the panels
    bottom_len = [panel.length for panel in panelS_tri[:N]]
    middle_len = [panel.length for panel in panelS_tri[N:-N]]
    top_len = [panel.length for panel in panelS_tri[-N:]]
    
    # angle of the panels called n*j (beta)
    bottom_angle = [panel.beta for panel in panelS_tri[:N]]
    middle_angle = [panel.beta for panel in panelS_tri[N:-N]]
    top_angle = [panel.beta for panel in panelS_tri[-N:]]
    
    # need to find the lift of the center of each panel individually and add them up
    
    # foil lift array
    bottom = numpy.empty(len(bottom_vt), dtype=float)
    for i in range (len(bottom_vt)):
        bottom[i]=-(P_inf+0.5*den*(u_inf**2-bottom_vt[i]**2))*bottom_len[i]*math.sin(bottom_angle[i])
        
    # flap lift array
    middle = numpy.empty(len(bottom_vt), dtype=float)
    for i in range (len(bottom_vt)):
        middle[i]=-(P_inf+0.5*den*(u_inf**2-middle_vt[i]**2))*middle_len[i]*math.sin(middle_angle[i])
        
    # flap lift array
    top = numpy.empty(len(bottom_vt), dtype=float)
    for i in range (len(bottom_vt)):
        top[i]=-(P_inf+0.5*den*(u_inf**2-top_vt[i]**2))*top_len[i]*math.sin(top_angle[i])
    
    print(bottom)
    print(middle)
    print(top)
    print(numpy.sum(bottom))
    print(numpy.sum(middle))
    print(numpy.sum(top))
    
    # summing the lift arrays and ading them up
    L=numpy.sum(bottom)+numpy.sum(middle)+numpy.sum(top)
    print(L)

In [ ]:
#print(panelS[:N])
#print(panelS[N:-N])
#print(panelS[-N:])
#print(panelS[32])

### Presssure Coefficient

In [ ]:
# plot it up
# plot surface pressure coefficient
pyplot.figure(figsize=(10, 6))
pyplot.grid()
pyplot.xlabel('$x$', fontsize=16)
pyplot.ylabel('$C_p$', fontsize=16)
pyplot.plot([panel.xc for panel in panelS_tri if panel.loc == 'upper'],
            [panel.cp for panel in panelS_tri if panel.loc == 'upper'],
            label='upper wing',
           color='b', linestyle='-', linewidth=2, marker='o', markersize=6)
pyplot.plot([panel.xc for panel in panelS_tri if panel.loc == 'lower'],
            [panel.cp for panel in panelS_tri if panel.loc == 'lower'],
            label='lower wing',
           color='r', linestyle='-', linewidth=2, marker='o', markersize=6)
pyplot.legend(loc='best', prop={'size':16})
pyplot.legend(loc='best', prop={'size':16})
#pyplot.xlim(-0.1, 1.5)
#pyplot.ylim(2.0, -10.0)
pyplot.title('Number of panels: {}'.format(len(panelS_tri)), fontsize=16);

In [ ]:
def alt_monplane(panelS, freestream, N):
    # find the source and vortex contributions
    A_source = source_contribution_normal(panelS)
    B_vortex = vortex_contribution_normal(panelS)
    
    # build the singularity matrix
    A = build_singularity_matrix_mono(A_source, B_vortex)
    b = build_freestream_rhs_mono(panelS, freestream)
    
    # solve for the singularity matrices
    strengths = numpy.linalg.solve(A,b)
    
    #store the strengths on each panel
    for i, panel in enumerate(panelS):
        panel.sigma = strengths[i]
        
    # store the circulation density
    gamma = strengths[-1]
    lift = gamma * sum(panel.length for panel in panelS)
    return lift